# Credit Risk Pipeline Quickstart

This notebook exercises the **Unified Risk Pipeline** end-to-end using the bundled synthetic dataset.
The sample includes stratified monthly observations, calibration hold-outs, stage-2 data and a future
scoring batch so that every major pipeline step can be validated quickly.

## 0. Environment setup

Install the latest development build of the pipeline (latest development build) directly from GitHub.
Re-run this cell if you refresh the kernel.

In [1]:
%pip install --quiet --no-cache-dir --upgrade --no-deps --force-reinstall "risk-pipeline[ml,notebook] @ git+https://github.com/selimoksuz/risk-model-pipeline.git@development"

Note: you may need to restart the kernel to use updated packages.


## 1. Imports and sample loader

The dataset ships with the package under `risk_pipeline.data.sample`.

In [2]:
from pathlib import Path
import pandas as pd

from risk_pipeline.core.config import Config
from risk_pipeline.unified_pipeline import UnifiedRiskPipeline
from risk_pipeline.data.sample import load_credit_risk_sample

sample = load_credit_risk_sample()
OUTPUT_DIR = Path('output/credit_risk_sample_notebook')
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

dev_df = sample.development
cal_long_df = sample.calibration_longrun
cal_recent_df = sample.calibration_recent
score_df = sample.scoring_future
data_dictionary = sample.data_dictionary

dev_df.head()

,app_id,customer_id,app_dt,snapshot_month,bureau_score,utilization_ratio,credit_usage_ratio,payment_income_ratio,balance_to_limit,monthly_spend,segment,region,employment_type,delinquent_last_6m,open_trades,channel_code,promo_flag,noise_feature,target
0,A031937001,319,2023-01-01,2023-01,654.0,1.000,0.878,0.267,0.929,1956.35,Retail,East,Contract,1,3,Online,0,0.7536,1
1,A029737002,297,2023-01-01,2023-01,602.0,0.971,0.853,0.288,0.900,1802.07,Retail,East,Salaried,1,5,Online,0,0.3251,1
2,A021237003,212,2023-01-01,2023-01,657.0,1.000,0.872,0.270,0.926,2169.97,Retail,East,Self-Employed,1,5,Branch,0,0.7342,1
3,A038237004,382,2023-01-01,2023-01,671.0,0.551,0.476,0.282,0.548,1806.44,Retail,South,Salaried,1,2,Online,0,0.8138,0
4,A023037005,230,2023-01-01,2023-01,654.0,0.432,0.380,0.208,0.396,1862.57,Retail,East,Self-Employed,0,4,Branch,0,0.1430,0


## 2. Quick sanity checks

In [3]:
dev_df['target'].value_counts(normalize=True).rename('default_rate')

0    0.72
1    0.28
Name: default_rate, dtype: float64

In [4]:
dev_df.groupby('snapshot_month')['target'].mean().rename('monthly_default_rate')

snapshot_month
2023-01    0.28
2023-02    0.28
2023-03    0.28
2023-04    0.28
2023-05    0.28
2023-06    0.28
2023-07    0.28
2023-08    0.28
2023-09    0.28
Name: monthly_default_rate, dtype: float64

## 3. Configure the pipeline

The configuration below enables dual modelling (raw + WoE), Optuna (single rapid trial), balanced model selection with stability guard rails,
noise sentinel monitoring, SHAP explainability, the WoE-LI and Shao logistic challengers, and the PD-constrained risk band optimizer.
Train/Test/OOT ratios and all threshold knobs (PSI/IV/Gini/Correlation) are explicit so the notebook mirrors production-ready configuration files.

In [5]:
cfg = Config(
    target_column='target',
    id_column='customer_id',
    time_column='app_dt',
    create_test_split=True,
    use_test_split=True,
    train_ratio=0.6,
    test_ratio=0.2,
    oot_ratio=0.2,
    stratify_test=True,
    oot_months=2,
    enable_dual=True,
    enable_tsfresh_features=True,
    enable_scoring=True,
    enable_stage2_calibration=True,
    output_folder=str(OUTPUT_DIR),
    selection_steps=['psi', 'univariate', 'iv', 'correlation', 'boruta', 'stepwise'],
    algorithms=[
        'logistic', 'gam', 'catboost', 'lightgbm', 'xgboost',
        'randomforest', 'extratrees', 'woe_boost', 'woe_li', 'shao', 'xbooster',
    ],
    model_selection_method='balanced',
    model_stability_weight=0.25,
    min_gini_threshold=0.45,
    max_train_oot_gap=0.08,
    psi_threshold=0.25,
    iv_threshold=0.02,
    univariate_gini_threshold=0.05,
    correlation_threshold=0.95,
    vif_threshold=5.0,
    woe_binning_strategy='iv_optimal',
    use_optuna=True,
    n_trials=1,
    optuna_timeout=120,
    hpo_method='optuna',
    hpo_trials=1,
    hpo_timeout_sec=120,
    use_noise_sentinel=True,
    calculate_shap=True,
    shap_sample_size=500,
    risk_band_method='pd_constraints',
    n_risk_bands=8,
    risk_band_min_bins=7,
    risk_band_max_bins=10,
    risk_band_micro_bins=1000,
    risk_band_min_weight=0.05,
    risk_band_max_weight=0.30,
    risk_band_hhi_threshold=0.15,
    risk_band_binomial_pass_weight=0.85,
    risk_band_alpha=0.05,
    risk_band_pd_dr_tolerance=1e-4,
    risk_band_max_iterations=100,
    risk_band_max_phase_iterations=50,
    risk_band_early_stop_rounds=10,
    calibration_stage1_method='isotonic',
    calibration_stage2_method='lower_mean',
    random_state=42,
)
cfg.model_type = 'all'


## 4. Run the unified pipeline

In [6]:
pipe = UnifiedRiskPipeline(cfg)
results = pipe.fit(
    dev_df,
    data_dictionary=data_dictionary,
    calibration_df=cal_long_df,
    stage2_df=cal_recent_df,
    score_df=score_df,
)

UNIFIED RISK PIPELINE EXECUTION

[Step 1/10] Data Processing...
  Added 40 tsfresh features
  Found 50 numeric and 5 categorical features

[Step 2/10] Data Splitting...
  train: 2240 samples, default rate: 28.12%
  test: 560 samples, default rate: 27.50%
  oot: 800 samples, default rate: 28.00%

[DUAL] Running RAW and WOE flows and selecting the best by AUC...

[Step 3/10] WOE Transformation & Univariate Analysis...
  WOE hesaplandi: 56 degisken

[Step 4/10] Feature Selection...
  Applying psi selection...
    Removing channel_code: oot psi 0.344 > 0.250
    Removing promo_flag: oot psi 0.501 > 0.250
    psi: 2 degisken cikarildi, 54 kaldi
  Applying univariate selection...
    Removing app_id: univariate gini 0.000 < 0.050
    Removing open_trades: univariate gini 0.014 < 0.050
    Removing noise_feature: univariate gini 0.031 < 0.050
    Removing bureau_score_std_tsfresh: univariate gini 0.047 < 0.050
    Removing utilization_ratio_std_tsfresh: univariate gini 0.038 < 0.050
    Remov

[I 2025-09-25 22:39:47,260] A new study created in memory with name: no-name-e6640018-0a69-4e13-87d2-2a463ae5752e


      Train AUC: 0.9564, OOT AUC: 0.9240, Test AUC: 0.9509, |Train-OOT Gini gap|: 0.0647
    Training RandomForest...


[I 2025-09-25 22:39:47,582] Trial 0 finished with value: 0.9369682042095836 and parameters: {'n_estimators': 144, 'max_depth': 10, 'min_samples_split': 40, 'min_samples_leaf': 14}. Best is trial 0 with value: 0.9369682042095836.
[I 2025-09-25 22:39:48,042] A new study created in memory with name: no-name-ac979d0c-3040-46f8-a322-156e30b12688


      Train AUC: 0.9644, OOT AUC: 0.9136, Test AUC: 0.9370, |Train-OOT Gini gap|: 0.1016
    Training ExtraTrees...


...

      Train AUC: 0.9813, OOT AUC: 0.9406, Test AUC: 0.9637, |Train-OOT Gini gap|: 0.0814
    Training CatBoost...


[I 2025-09-25 22:40:11,560] Trial 0 finished with value: 0.9573123920414561 and parameters: {'iterations': 144, 'depth': 10, 'learning_rate': 0.22227824312530747, 'l2_leaf_reg': 6.387926357773329}. Best is trial 0 with value: 0.9573123920414561.
[I 2025-09-25 22:40:11,856] A new study created in memory with name: no-name-c1c8a26d-57a8-4864-a1d4-46f838f9ad48


      Train AUC: 0.9900, OOT AUC: 0.9292, Test AUC: 0.9573, |Train-OOT Gini gap|: 0.1217
    Training GAM...


[I 2025-09-25 22:40:12,253] Trial 0 finished with value: 0.9667967500479816 and parameters: {'n_splines': 12, 'lam': 6.351221010640703}. Best is trial 0 with value: 0.9667967500479816.


      Train AUC: 0.9689, OOT AUC: 0.9430, Test AUC: 0.9668, |Train-OOT Gini gap|: 0.0518
    Training WoeBoost...
      Train AUC: 0.9902, OOT AUC: 0.9338, Test AUC: 0.9585, |Train-OOT Gini gap|: 0.1127
    Training WoeLogisticInteraction...
      Train AUC: 0.9688, OOT AUC: 0.9414, Test AUC: 0.9655, |Train-OOT Gini gap|: 0.0550
    Training ShaoLogit...
      Train AUC: 0.9672, OOT AUC: 0.9429, Test AUC: 0.9663, |Train-OOT Gini gap|: 0.0486
    Best model: LogisticRegression (OOT AUC: 0.9431, method: balanced), |train-oot gini gap|: 0.0483

[Step 6/10] Stage 1 Calibration...
  Added 40 tsfresh features
  Found 50 numeric and 5 categorical features
    Applying Stage 1 calibration (method=isotonic)...
      Stage 1 target (long-run mean) default rate: 26.00%
      ECE: 0.0000
      MCE: 0.0000
      Brier Score: 0.1924

[Step 7/10] Stage 2 Calibration...
  Added 40 tsfresh features
  Found 50 numeric and 5 categorical features
    Applying Stage 2 calibration (method=lower_mean)...
   

## 5. Inspect key outputs

In [7]:
best_model = results.get('best_model_name')
model_scores = results.get('model_results', {}).get('scores', {})
print(f'Best model: {best_model}')
pd.DataFrame(model_scores).T

Best model: LogisticRegression


,train_auc,test_auc,oot_auc,train_gini,test_gini,oot_gini,train_oot_gap
LogisticRegression,0.967220,0.966605,0.943092,0.934440,0.933210,0.886184,0.048256
RandomForest,0.965637,0.958416,0.931769,0.931274,0.916832,0.863537,0.067737
ExtraTrees,0.959496,0.952834,0.927343,0.918992,0.905668,0.854686,0.064306
LightGBM,0.996461,0.947748,0.921243,0.992921,0.895496,0.842487,0.150435
XGBoost,0.981282,0.963694,0.940589,0.962563,0.927388,0.881177,0.081386
CatBoost,0.990044,0.957312,0.929219,0.980089,0.914625,0.858437,0.121652
GAM,0.968939,0.966797,0.943045,0.937878,0.933594,0.886091,0.051787
WoeBoost,0.990165,0.958512,0.933807,0.980329,0.917024,0.867614,0.112716
WoeLogisticInteraction,0.968841,0.965485,0.941356,0.937681,0.930971,0.882712,0.054969
ShaoLogit,0.967222,0.966333,0.942945,0.934443,0.932666,0.885889,0.048554


In [8]:
feature_report = pipe.reporter.reports_.get('features')
feature_report.head() if feature_report is not None else 'No feature report available.'

,feature,raw_feature,description,category,iv,gini_raw,gini_woe,gini_drop,is_tsfresh,tsfresh_source,...,importance_LogisticRegression,importance_RandomForest,importance_ExtraTrees,importance_LightGBM,importance_XGBoost,importance_CatBoost,importance_GAM,importance_WoeBoost,importance_WoeLogisticInteraction,importance_ShaoLogit
0,bureau_score_min_tsfresh,bureau_score_min_tsfresh,,,1.673774,-0.638329,0.626627,-1.264956,True,bureau_score,...,2.058399,0.488459,0.424839,1804,0.214816,31.604701,0.142857,4633.538953,0.142857,0.142857
1,balance_to_limit,balance_to_limit,Risk - Balance to credit limit ratio,Risk,0.479553,0.373962,0.369513,0.004449,False,NaN,...,2.168241,0.140603,0.094083,2153,0.102941,15.030887,0.142857,2123.980738,0.142857,0.142857
2,delinquent_last_6m_mean_tsfresh,delinquent_last_6m_mean_tsfresh,,,0.368268,0.235059,0.235059,0.000000,True,delinquent_last_6m,...,2.328630,0.130299,0.181816,261,0.294976,15.425114,0.142857,1585.364357,0.142857,0.142857
3,segment,segment,Demographic - Customer segment,Demographic,0.316595,0.287406,0.276605,0.010802,False,NaN,...,2.051961,0.098429,0.130496,385,0.132571,11.402640,0.142857,1100.313803,0.142857,0.142857
4,channel_code,channel_code,Operational - Acquisition channel (drift driver),Operational,0.200877,0.197491,0.187716,0.009775,False,NaN,...,3.029687,0.074082,0.100574,221,0.119677,12.514840,0.142857,1037.230346,0.142857,0.142857


In [9]:
calibration_report = pipe.reporter.reports_.get('calibration')
calibration_report

{'stage1': {'method': 'isotonic',
  'metrics': {'ece': 1.1098265164020762e-16,
   'mce': 1.1102230246251565e-16,
   'brier': 0.1923758485173277,
   'log_loss': 0.5729493571843972,
   'mean_predicted': 0.2600000000000002,
   'mean_actual': 0.26,
   'calibration_gap': 1.6653345369377348e-16},
  'details': {'method': 'isotonic', 'long_run_rate': 0.26, 'base_rate': 0.26}},
 'stage2': {'method': 'lower_mean',
  'metrics': {'ece': 1.6653345369377348e-16,
   'mce': 1.6653345369377348e-16,
   'brier': 0.22440000000000004,
   'log_loss': 0.6410354778811554,
   'mean_predicted': 0.3400000000000002,
   'mean_actual': 0.34,
   'calibration_gap': 1.6653345369377348e-16},
  'details': {'method': 'lower_mean',
   'recent_rate': 0.34,
   'stage1_rate': 0.2600928903179708,
   'target_rate': 0.34,
   'adjustment_factor': 1.3072252747252744,
   'achieved_rate': 0.3400000000000002}}}

In [10]:
risk_bands = pipe.reporter.reports_.get('risk_bands_summary', {})
risk_bands

{'Risk Bands Summary': ['Herfindahl Index: 0.0000',
  'Entropy: 0.0000',
  'Gini Coefficient: 0.0000',
  'Hosmer-Lemeshow p-value: 0.0000',
  'KS Statistic: nan']}

## 6. Generated files

In [11]:
sorted(p.relative_to(OUTPUT_DIR.parent) for p in OUTPUT_DIR.glob('**/*') if p.is_file())

[WindowsPath('credit_risk_sample_notebook/best_model_raw_20250925_164952.joblib'),
 WindowsPath('credit_risk_sample_notebook/best_model_raw_20250925_200130.joblib'),
 WindowsPath('credit_risk_sample_notebook/best_model_raw_20250925_223855.joblib'),
 WindowsPath('credit_risk_sample_notebook/best_model_raw_20250925_224013.joblib'),
 WindowsPath('credit_risk_sample_notebook/final_features_20250925_164952.json'),
 WindowsPath('credit_risk_sample_notebook/final_features_20250925_200130.json'),
 WindowsPath('credit_risk_sample_notebook/final_features_20250925_223855.json'),
 WindowsPath('credit_risk_sample_notebook/final_features_20250925_224013.json'),
 WindowsPath('credit_risk_sample_notebook/final_model_20250925_164952.joblib'),
 WindowsPath('credit_risk_sample_notebook/final_model_20250925_200130.joblib'),
 WindowsPath('credit_risk_sample_notebook/final_model_20250925_223855.joblib'),
 WindowsPath('credit_risk_sample_notebook/final_model_20250925_224013.joblib'),
 WindowsPath('credit_ris

## 7. XBooster scorecard

In [12]:
xbooster_artifacts = results.get('model_results', {}).get('interpretability', {}).get('XBooster', {})
if isinstance(xbooster_artifacts, dict):
    scorecard_df = xbooster_artifacts.get('scorecard_points')
    warnings = xbooster_artifacts.get('warnings')
    display_obj = scorecard_df.head() if hasattr(scorecard_df, 'head') else xbooster_artifacts
else:
    warnings = None
    display_obj = 'No XBooster artifacts available.'
print('Warnings:', warnings if warnings else 'None')
display_obj

Warnings: None


{}

## 8. Automating via script

`examples/quickstart_demo.py` mirrors the steps above so the flow can be validated headless
(e.g. in CI pipelines).